# <Section 01 Project: 다음 분기에 어떤 게임을 설계해야 할까?>


In [ ]:
#다음 분기에 출고량이 높을 것이라 예상되는 게임

## 1. EDA

In [ ]:
import pandas as pd

df = pd.read_csv("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv")
df.drop(["Unnamed: 0"], axis = 1, inplace = True)
df.head()

In [ ]:
df.info()

In [ ]:
import numpy as np

def int_to_year(num):
    this_year = 21 #2021년도
    if num < 0 or num == np.nan:
        return num.replace(num, np.nan)
    elif num < this_year:
        return int(num) + 2000
    elif num < 100:
        return int(num) + 1900
    else:
        return int(num)

def convert_str_to_number(string):
    if "K" in string:
        return float(string.replace("K", "")) * (10**3)
    elif "M" in string:
        return float(string.replace("M", "")) * (10**6)
    else:
        return float(string) * (10**6)

df.dropna(axis = 0, inplace = True) #결측치 제거

df["Year"] = df["Year"].apply(int_to_year)
for Sales in ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]:
    df[Sales] = df[Sales].apply(convert_str_to_number)

df.reset_index(drop = True, inplace = True)

#확인
df.info()
print()
df

## 2. 분석

### 1. "**지역**"과 "**선호하는 게임장르**"는 연관이 있을것인가?
= 선호하는 게임장르는 곧 출고량과 직결되므로 위 물음을 답하기 위해 Chi-square test을 사용할 수 있음.

    (1) 귀무가설: 지역과 게임장르는 출고량에 대해서 서로 연관이 있을 것이다.
    (2) 대안가설: 지역과 게임장르는 출고량과 대해서 서로 연관이 없을 것이다.
    (3) 신뢰도: 95 %

In [ ]:
df_Genre_sales = df.drop(["Name", "Platform", "Year", "Publisher"], axis = 1, inplace = False)
df_Genre_sales

In [ ]:
Genre_Region_sum_list = []
for Sales in ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]:
    Genre_sum = df_Genre_sales[Sales].groupby(df_Genre_sales["Genre"]).sum()
    Genre_Region_sum_list.append(Genre_sum)

df_Genre_Region_sum = pd.concat(Genre_Region_sum_list, axis = 1)
df_Genre_Region_sum

In [ ]:
from scipy import stats

chi1 = stats.chi2_contingency(df_Genre_Region_sum, correction = False)
pv1 = chi1[1]
pv1

In [ ]:
#p-value가 0.05 이하이므로 귀무가설을 기각, 대안가설을 채택함.
#즉, 지역과 게임 장르는 출고량에 대해서 서로 연관이 없다고 볼 수 있다.

1. 연도별로 게임 장르의 트렌드를 통해 다음 분기에 설계할 게임의 장르를 추릴 수 있을 것이다.

    (1) 연도별 - 지역별로 출고량의 합 구하기 및 비율 구하기
        연도별로 특정 지역의 전체 출고량(그 지역의 게임시장의 크기)를 알 수 있음.

    (2) 연도별 - 장르별로 출고량의 합 구하기
        연도별로 어떤 장르의 출고량이 가장 높은지 알 수 있음.

    (3) 장르별 - 지역별로 출고량의 합 구하기
        





In [ ]:
df_year_sales = df.drop(["Name", "Platform", "Genre", "Publisher"], axis = 1, inplace = False)

In [ ]:
Year_Region_sum_list = []
for Sales in ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]:
    Year_sum = df_year_sales[Sales].groupby(df_year_sales["Year"]).sum()
    Year_Region_sum_list.append(Year_sum)

df_Year_Region_sum = pd.concat(Year_Region_sum_list, axis = 1)
df_Year_Region_sum["All_Sales"] = df_Year_Region_sum.sum(axis = 1)

df_Year_Region_All_Sales = df_Year_Region_sum["All_Sales"]

df_Year_Region_sum.head()

In [ ]:
for Sales in ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]:
    df_Year_Region_sum[Sales] = df_Year_Region_sum[Sales] / df_Year_Region_sum["All_Sales"]

df_Year_Region_sum.drop(["All_Sales"], axis = 1, inplace = True)
df_Year_Region_sum.drop([2017, 2020], axis = 0, inplace = True) #데이터 수 적어 신뢰성 부족하다고 판단.
df_Year_Region_sum = df_Year_Region_sum * 100
df_Year_Region_sum.head()

In [ ]:
import matplotlib.pyplot as plt

df_Year_Region_sum.plot(kind = "bar", stacked = True, fontsize = 9, figsize = (13, 8))
plt.xticks(rotation = 40)
plt.axhline(y = 70, color = "r", linewidth = 1)

plt.xlabel("Year")
plt.ylabel("Market Share (%)")
plt.title("< Market share by continent (country) >", fontsize = 12)

plt.show()

In [ ]:
#위 그래프는 게임의 장르에 상관없이 연도별로 각 지역별 출고량을 누적 막대 그래프로 나타낸 자료 입니다.
# >> 북미 + 유럽지역의 출고량 압도적임.

In [ ]:
#연도별 게임장르의 트렌드 분석

In [ ]:
df["All_Sales"] = df["NA_Sales"] + df["EU_Sales"] + df["JP_Sales"] + df["Other_Sales"]
df_Year_Genre = df.drop(["Name", "Platform", "Publisher", "NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"], axis = 1, inplace = False)
df_Year_Genre.head()

In [ ]:
Year_Genre_sum = df_Year_Genre.groupby(["Year", "Genre"])["All_Sales"].sum()
Year_Genre_ratio = Year_Genre_sum / df_Year_Region_All_Sales
df_Year_Genre_ratio = pd.DataFrame(Year_Genre_ratio)
df_Year_Genre_ratio.drop([2017, 2020], axis = 0, inplace = True) #데이터 수 적어 신뢰성 부족하다고 판단.
df_Year_Genre_ratio = df_Year_Genre_ratio * 100 #백분율

In [ ]:
colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:gray", "tab:olive", "tab:cyan", "tan", "slateblue"]

df_Year_Genre_ratio.unstack().plot(kind = 'line',
                                   fontsize = 8,
                                   figsize = (7, 13),
                                   color = colors,
                                   subplots = True,
                                   ylim = [0, 40],
                                   grid = True,
                                   xlabel = "Year",
                                   ylabel = "Share(%)")

plt.show()

In [ ]:
df_Year_Genre_sum = pd.DataFrame(Year_Genre_sum)

Year_Genre_count = df_Year_Genre.groupby(["Year", "Genre"])["Genre"].count()
df_Year_Genre_count = pd.DataFrame(Year_Genre_count)

df_Year_Genre_sum["Count"] = df_Year_Genre_count["Genre"]

df_Year_Genre_sum.reset_index(level = 0, drop = True, inplace = True)

#확인
df_Year_Genre_sum

In [ ]:
from sklearn.preprocessing import StandardScaler
import seaborn as sns

labels = df_Year_Genre_sum.index

standard_df_Year_Genre = StandardScaler().fit_transform(df_Year_Genre_sum)
df_standard_Year_Genre = pd.DataFrame(standard_df_Year_Genre, columns = ["All_Sales", "Count"])

df_standard_Year_Genre["Labels"] = labels

#시각화
sns.lmplot(x = 'Count',
           y = 'All_Sales',
           hue = 'Labels',
           #col = 'Labels',
           line_kws={'label':"tip"},
           data = df_standard_Year_Genre,
           #col_wrap = 3,
           order = 1)

In [ ]:
#나라별로 보다 선호하는 장르
#나라별로 보다 선호하는 플랫폼
#위에 나온 그래프는 전체 지역에서 선호하는 장르를 추린것임.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

df_Total_Sales = df[["All_Sales"]]
scaler = StandardScaler()
df_Total_Sales_standard = scaler.fit_transform(df_Total_Sales)

sum_of_squared_distances = []
K = range(1, 15)
for k in K:
    km = KMeans(n_clusters = k)
    km = km.fit(df_Total_Sales_standard)
    sum_of_squared_distances.append(km.inertia_)

plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()


In [ ]:
scaler = StandardScaler()
All_Sales_std = scaler.fit_transform(df[["All_Sales"]])
df_All_Sales_std = pd.DataFrame(All_Sales_std, columns = ["std_All_Sales"])

kmeans = KMeans(n_clusters = 6, random_state = 7)
kmeans.fit(df_All_Sales_std)
K_labels = kmeans.labels_
df_cluster = pd.DataFrame(K_labels, columns = ["Sale_rating"])

df["std_All_Sales"] = df_All_Sales_std["std_All_Sales"]
df["Sale_rating"] = df_cluster["Sale_rating"]
df.head()

In [ ]:
#scatterplot 그리기
sns.scatterplot(x = "Year", y = "std_All_Sales", hue = "Sale_rating", data = df)

In [ ]:
df_Sales_high = df.query("Sale_rating > 0") #상위 14.57 %
df_Sales_low = df.query('Sale_rating == 0') #상위 85.43 %

#높은 출고율이 전체 출고율에 차지하는 비율
rate = (df_Sales_high.shape[0] / (df_Sales_high.shape[0] + df_Sales_low.shape[0])) * 100

print("'높은 출고량'에 대한 기준을 출고량의 상위", round(rate, 2), "% 로 두었습니다. \n")

#EDA
df_Sales_high.reset_index(drop = True, inplace = True)
df_Sales_high = df_Sales_high.drop(["Sale_rating", "std_All_Sales", "Name", "Publisher"], axis = 1, inplace = False)
df_Sales_high

In [ ]:
df_high_Year_Plat_count = pd.crosstab(df_Sales_high.Year, df_Sales_high.Platform, margins=True)
df_high_Year_Plat_count.head()

In [ ]:
import seaborn as sns

df_high_Year_Plat_count_drop = df_high_Year_Plat_count.drop(["All"], axis = 0) #마지막 row에 있는 All index를 없앰

for Plat in df_high_Year_Plat_count_drop.columns:
    df_high_Year_Plat_count_drop[Plat] = df_high_Year_Plat_count_drop[Plat] / df_high_Year_Plat_count_drop["All"]

df_high_Year_Plat_count_drop2 = df_high_Year_Plat_count_drop.drop(["All"], axis = 1)
df_high_Year_Plat_ratio = round(df_high_Year_Plat_count_drop2, 2).T
df_high_Year_Plat_ratio_drop = df_high_Year_Plat_ratio.drop(df_high_Year_Plat_ratio.columns[:25], axis = 1)


plt.rcParams["figure.figsize"] = (30, 10)
sns.heatmap(df_high_Year_Plat_ratio_drop,
           annot = True, #실제 값 화면에 나타내기
           cmap = 'Greens', #색상
           vmin = 0, vmax = 0.5 , #컬러차트 영역
           )

In [ ]:
#필요없는 platform제거 (다음분기에 설계할 게임의 플랫폼의 후보에서 제외)
df_high_Year_Plat_ratio_drop2 = df_high_Year_Plat_ratio_drop.drop(["2600", "DC", "GB", "GBA", "GC", "GEN", "N64", "NES", "PS", "PSP", "PSV", "SAT", "SCD", "SNES", "XB"], axis = 0, inplace = False)

plt.rcParams["figure.figsize"] = (30, 10)
sns.heatmap(df_high_Year_Plat_ratio_drop2,
           annot = True, #실제 값 화면에 나타내기
           cmap = 'Greens', #색상
           vmin = 0, vmax = 0.5 , #컬러차트 영역
           )

plt.show()
## 출고량이 높은 게임들이 어떤 플랫폼을 이용해 만들어 졌는지 분석
# - 기술이 발전함에 따라 게임 플랫폼도 점점 발전하기 때문에 2005년 이전 데이터들은 필요없다고 판단. (2005년도 부터 분석)
#3DS >> 출고량이 높았던 게임들 중 2011년부터 2016년까지  약 6 ~ 10% 가 3DS 플랫폼 기반의 게임이다.
#DS >> 출고량이 높았던 게임들 중 2005년부터 2010년까지 약 10 ~ 20 %가 DS 플랫폼 기반의 게임이다.
#PC >> 출고량이 높았던 게임들 중 2005년부터 2015년까지 약 2 ~ 5% 가 PC 플랫폼 기반의 게임이다.
#PS Series >> PS Platform이 간헐적으로 업데이트를 하고 있으며 14년도 이후에 출고량이 높았던 게임들 중 가장 많은 비중을 차지한 게임이 PS4 플랫폼 기반의 게임이었음. (14년도 29%, 15년도 41%, 16년도 72%)
#XOne >> 출고량이 높았던 게임들 중 2013년부터 2016년까지 약 10 ~ 25 %가 XOne 플랫폰 기반의 게임이다.

# >> 1순위 PS4, 2순위, XONE, 3순위 WiiU, 3DS

In [ ]:
df_high_Year_Genre_count = pd.crosstab(df_Sales_high.Year, df_Sales_high.Genre, margins=True)
df_high_Year_Genre_count.head()

In [ ]:
import seaborn as sns

df_high_Year_Genre_count_drop = df_high_Year_Genre_count.drop(["All"], axis = 0) #마지막 row에 있는 All index를 없앰

for Plat in df_high_Year_Genre_count_drop.columns:
    df_high_Year_Genre_count_drop[Plat] = df_high_Year_Genre_count_drop[Plat] / df_high_Year_Genre_count_drop["All"]

df_high_Year_Genre_count_drop2 = df_high_Year_Genre_count_drop.drop(["All"], axis = 1)
df_high_Year_Genre_ratio = round(df_high_Year_Genre_count_drop2, 2).T
df_high_Year_Genre_ratio_drop = df_high_Year_Genre_ratio.drop(df_high_Year_Genre_ratio.columns[:25], axis = 1)


plt.rcParams["figure.figsize"] = (30, 10)
sns.heatmap(df_high_Year_Genre_ratio_drop,
           annot = True, #실제 값 화면에 나타내기
           cmap = 'Greens', #색상
           vmin = 0, vmax = 0.3 , #컬러차트 영역
           )

In [ ]:
# 1순위 - Shooter, 2순위 - Action, Sports, 3순위 - Role-Playing

In [ ]:
df_high_Plat_Genre_count = pd.crosstab(df_Sales_high.Platform, df_Sales_high.Genre, margins=True)
df_high_Plat_Genre_count.head()

In [ ]:
#관심 있는 row, column만 추출
Genre_drop_list = []
for i in df_high_Plat_Genre_count.columns:
    if i not in ["Action", "Shooter", "Sports", "Role-Playing"]:
        Genre_drop_list.append(i)
    else:
        continue

Plat_drop_list = []
for j in df_high_Plat_Genre_count.index:
    if j not in ["PS4", "XOne", "WiiU", "3DS"]:
        Plat_drop_list.append(j)
        

df_high_Plat_Genre_F1 = df_high_Plat_Genre_count.drop(Genre_drop_list, axis = 1)
df_high_Plat_Genre_F2 = df_high_Plat_Genre_F1.drop(Plat_drop_list, axis = 0)
df_high_Plat_Genre_F2

In [ ]:
from scipy import stats

chi1 = stats.chi2_contingency(df_high_Plat_Genre_F2, correction = False)
pv1 = chi1[1]
pv1